# Manejo de información en Python

En este cuaderno mostraremos distintas maneras para manejar información en las que se puede usar Python para diversos formatos.


In [ ]:
import pandas as pd
import json
import yaml
from zipfile import ZipFile


## Archivos zip


Podemos trabajar con archivos zip con la libreria zipfile 

Podemos listar los archivos 

In [ ]:
with ZipFile("datos.zip", 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()

File Name                                             Modified             Size
archivo_json.json                              2023-03-03 15:56:24          164
archivo_texto.txt                              2023-03-03 15:53:36           30
archivo_yaml.yaml                              2023-03-03 15:59:06          341
penguins.csv                                   2023-03-03 00:35:00        15241
penguins.parquet                               2023-03-03 16:01:00         8812


Podemos extraer todos los archivos

In [ ]:
with ZipFile("datos.zip", 'r') as zip:
  zip.extract("penguins.csv")

Podemos extraer un solo archivo 



In [ ]:
with ZipFile("datos.zip", 'r') as zip:
    zip.extractall()

## Archivos básicos

El ejemplo más común y básico es utilizar archivos de texto, los cuales se pueden abrir con la herramienta open y así acceder a su contenido.

In [ ]:
with open('archivo_texto.txt',"r") as f:
    read_data = f.read()
read_data

'primera linea\nsegunda linea\n'

De la misma manera es posible escribir contenido en archivos y guardarlo para posterior uso.


In [ ]:
with open('archivo_texto.txt',"a") as f:
  f.write('Tercera linea\n')
  
with open('archivo_texto.txt',"r") as f:
    for linea in f:
      print(linea, end='')

primera linea
segunda linea
Tercera linea
Tercera linea
Tercera linea


Uno de los formatos soportados de manera nativa por Python es json. En el cual se abre el archivo de la misma manera que en formatos de texto y se lee el contenido con la función load. Lo que obtenemos de esta función es un diccionario de datos con el contenido del archivo.

In [ ]:
with open('archivo_json.json',"r") as f:
  datos_json = json.load(f)
datos_json

Con esto podemos acceder a elementos particulares del diccionario obtenido, así también podemos editar valores para después guardarlos en disco.

In [ ]:
print(datos_json["nombre"])


In [ ]:
datos_json["amigos"] = ["Solin","Luigi Geovany"]
with open('archivo_json_2.json',"w") as f:
  json.dump(datos_json,f)

Un formato que ha ganado popularidad recientemente es el formato yaml el cual tiene la característica detener una mejor legibilidad por humanos.



In [ ]:
with open('archivo_yaml.yaml',"r") as f:
  datos_yaml = yaml.safe_load(f)
datos_yaml

A través de la librería homónima, se puede obtener de este un diccionario de datos de la misma manera que json. Con la información la cual puede ser accedida y modificada. yaml también cuenta con una función para escribir en archivos de la misma manera que json.

In [ ]:
datos_yaml['published']

In [ ]:
datos_yaml['autor'] = 'Irivng'

with open('archivo_yaml_2.yaml',"w") as f:
  yaml.dump(datos_yaml,f)

## Uso de pandas 

Uno de los formatos más ampliamente utilizados para almacenar información es el formato csv. Aunque puede ser leído de la misma manera como un archivo de texto la librería panda nos ayuda a obtener estos archivos y analizarlos de manera sencilla.

In [ ]:
datos = pd.read_csv("penguins.csv")

Pandas soporta también el acceso a otros formatos más especializados como es parquet, orc, Excel, stata, sas, html e incluso SQL.

In [ ]:
df_pinguinos = pd.read_parquet("penguins.parquet")
df_pinguinos

El objeto obtenido de leer una fuente por pandas es llamado dataframe.
Las columnas de datos pueden ser seleccionadas de manera sencilla al establecer las variables necesarias.


In [ ]:
df_pinguinos_textos = df_pinguinos[['species', 'island','sex']]
df_pinguinos_textos

También es posible filtrar los registros respecto a uno o varios valores de columnas. Por ejemplo, en esta tabla nos concentramos sólo en los elementos con masa mayor a 4000.

In [ ]:
df_pinguinos_gordos = df_pinguinos[df_pinguinos["body_mass_g"] > 4000]
df_pinguinos_gordos = df_pinguinos[df_pinguinos["body_mass_g"] > 4000][['species', 'island']]
df_pinguinos_gordos

La manera en que funciona esto es que primero se calcula un vector booleano donde verdadero son los datos incluidos en la tabla, y falso son los que serán omitidos.

In [ ]:
df_pinguinos["body_mass_g"] > 4000

Las tablas de datos permiten también la creación de gráficos de manera rápida. Aunque para un control más fino y gráficos detallados, se recomiendan otras herramientas como matplotlib y seaborn.

In [ ]:
df_pinguinos.plot.scatter(x="bill_depth_mm", y="body_mass_g", alpha=0.5)

Podemos calcular nuevas variables en base a las existentes a través de operaciones básicas que se realizan de manera eficiente. no se recomienda el uso en pandas de ciclos para atravesar la estructura de la tabla.

In [ ]:
df_pinguinos["bill_area_mm2"] = (df_pinguinos["bill_length_mm"] * df_pinguinos["bill_depth_mm"]	) / 2
df_pinguinos

También se tiene fácil acceso a funciones de agregación, por ejemplo la media la cual es calculada para las variables pertinentes.

In [ ]:
df_pinguinos.mean()

Una función muy útil es la de describe, la cual nos da una serie de agregaciones distintas para todas las variables disponibles.

In [ ]:
df_pinguinos.describe()

Si es de nuestro interés calcular agregaciones por clases es también posible utilizando groupby.

In [ ]:
df_pinguinos.groupby(["species","island"]).mean()

Podemos tambien ordenar los registros respecto a alguna variable de interés.

In [ ]:
df_pinguinos_gordos.sort_values("body_mass_g")

En algunos casos es necesario cambiar la estructura de la tabla y convertir, por ejemplo, valores y agregaciones de filas en columnas. Esto se logra a través de pivot_table.

In [ ]:
df_pinguinos.pivot_table(
    values="bill_length_mm", 
    index="year", 
    columns="species", 
    aggfunc="mean",
    margins=True
)

El caso inverso es también posible al convertir valores de columnas en valores de registros a través de la función melt.

In [ ]:
df_pinguinos[["species",	"island",	"bill_length_mm",	"bill_depth_mm",	"flipper_length_mm"]].melt(
    id_vars=["species",	"island",]
)